In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import to_categorical

import tensorflow as tf

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# Loading the dataset
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [3]:
y_test_one_hot = to_categorical(y_test)
y_train_one_hot = to_categorical(y_train)

In [4]:
y_test_one_hot.shape, y_train_one_hot.shape

((25000, 2), (25000, 2))

In [5]:
# Preprocessing sestences
max_review_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_len)

## Building the model
Pipeline
- Define X and y as tf.placeholders
- Helper functions for weights and biases as tf.Variables
- Define Hyper parameters
    - Learning Rate
    - Batch size
    - Number of epochs
    - Embedding vector size
    - Keep Probability
- Build the model
    - Embedding layer (None, sequence_length, embedding_vector_size, 1) + Dropout
    - CONV_1d_2 (2, embedding_vector_size, 32) with 32 kernels + Leaky Relu + MaxPool_1d_2 + Dropout
    - CONV_1d_2 (2, 32, 32) with 32 kernels + Leaky Relu + MaxPool_1d_2 + Dropout
    - Dense Layer (??, 256) with relu activation + Dropout
    - Dense Layer (256, 1) with sigmoid activation
- Define the loss operation - Binary crossentropy
- Define the optimizer operation - ADAM
- Run the training loop
- Capture summaries for visualization

In [6]:
logdir = 'log/'

VOCAB_SIZE = 5000
EMBEDDING_SIZE = 32
EPOCHS = 3
BATCH_SIZE = 64
KEEP_PROB = 0.75

In [7]:
X = tf.placeholder(dtype=tf.int32)
y = tf.placeholder(dtype=tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

In [8]:
def weight_variable(shape, name='weights'):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init, name=name)

def bias_variable(shape, name='bias'):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init, name=name)

In [9]:
# Building the model
with tf.name_scope('embedding'):
    W = weight_variable([VOCAB_SIZE, EMBEDDING_SIZE])
    embedded_chars = tf.nn.embedding_lookup(W, X)

with tf.name_scope('embedding_dropout'):
    embedded_drop = tf.nn.dropout(embedded_chars, keep_prob)
    
with tf.name_scope('conv_maxpool_1'):
    W = weight_variable([2, EMBEDDING_SIZE, 32])
    b = bias_variable([32])
    h_conv1 = tf.nn.leaky_relu(tf.nn.conv1d(embedded_drop, W, 1, padding='VALID') + b, alpha=0.1)
    h_pool1 = tf.layers.max_pooling1d(h_conv1, 2, 2, padding='VALID')

with tf.name_scope('conv_maxpool_1_dropout'):
    h_pool1_drop = tf.nn.dropout(h_pool1, keep_prob)

with tf.name_scope('conv_maxpool_2'):
    W = weight_variable([2, 32, 32])
    b = bias_variable([32])
    h_conv2 = tf.nn.leaky_relu(tf.nn.conv1d(h_pool1_drop, W, 1, padding='VALID') + b, alpha=0.1)
    h_pool2 = tf.layers.max_pooling1d(h_conv2, 2, 2, padding='VALID')

with tf.name_scope('fully_connected'):
    W = weight_variable([3968, 256])
    b = bias_variable([256])
    
    pool_flatten = tf.reshape(h_pool2, [-1, 3968])
    
    pool_flatten_drop = tf.nn.dropout(pool_flatten, keep_prob)
    
    fc1 = tf.nn.relu(tf.matmul(pool_flatten_drop, W) + b)

with tf.name_scope('fc1_dropout'):
    fc1_drop = tf.nn.dropout(fc1, keep_prob)

with tf.name_scope('output'):
    W = weight_variable([256, 2])
    b = bias_variable([2])
    
    y_ = tf.matmul(fc1_drop, W) + b

In [10]:
# Defining the loss op and trainop
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
tf.summary.scalar('loss', loss_op)

train_op = tf.train.AdamOptimizer().minimize(loss_op)

# Evaluations
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Summaries
merged = tf.summary.merge_all()

# Init op
init_op = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
# Training loop
with tf.Session() as sess:
    
    # Summary Writer
    summary_writer = tf.summary.FileWriter(logdir, sess.graph)
    
    sess.run(init_op)
    
    for epoch in range(EPOCHS):
        print('Epoch {}'.format(epoch + 1))
        
        for index, offset in tqdm(list(enumerate(range(0, X_train.shape[0], BATCH_SIZE))), ncols=100):
            xs, ys = X_train[offset: offset + BATCH_SIZE], y_train_one_hot[offset: offset + BATCH_SIZE]
            summary, _ = sess.run([merged, train_op], feed_dict={
                X: xs,
                y: ys,
                keep_prob: KEEP_PROB
            })
            
            summary_writer.add_summary(summary, epoch)
            
        train_accuracy = accuracy.eval(feed_dict={
            X: X_train,
            y: y_train_one_hot,
            keep_prob: KEEP_PROB
        })
        
        test_accuracy = accuracy.eval(feed_dict={
            X: X_test,
            y: y_test_one_hot,
            keep_prob:KEEP_PROB
        })

        print('Training Accuracy: {}%\nTest Accuracy: {}%'.format(train_accuracy, test_accuracy))
    
    summary_writer.close()

Epoch 1


100%|█████████████████████████████████████████████████████████████| 391/391 [00:50<00:00,  7.69it/s]


Training Accuracy: 0.509440004825592%
Test Accuracy: 0.5073599815368652%
Epoch 2


100%|█████████████████████████████████████████████████████████████| 391/391 [00:57<00:00,  6.86it/s]


Training Accuracy: 0.6567599773406982%
Test Accuracy: 0.6414399743080139%
Epoch 3


100%|█████████████████████████████████████████████████████████████| 391/391 [01:02<00:00,  6.24it/s]


Training Accuracy: 0.8992400169372559%
Test Accuracy: 0.8668000102043152%
